In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so    
import sqlite3
from config import file_directory, dropbox_directory

Load data and do calculations for unitcosts

In [4]:
# Load base CSV. Convert to ISO 8859 otherwise problems with regions/languages in datetime fields
file_name = "Sample-Superstore.csv"
file_path = fr'{dropbox_directory}\{file_name}'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

In [5]:
# column/data conversions

# Calculate PricePreDiscount - with it you can reverse-calculate sales and others via SQL query which is supposed to be more efficient
df['PricePreDiscount'] = df['Sales']                                                    
df.loc[df['Discount'] != 0,'PricePreDiscount'] = df['Sales'] / (1- df['Discount'])      
df['PricePreDiscount_NEW'] = (df['Sales'] / (1- df['Discount'])) / df['Quantity']       
current_index = df.columns.get_loc('Sales')
desired_index = 19
df.insert(desired_index, 'Sales', df.pop('Sales'))
current_index = df.columns.get_loc('PricePreDiscount')
desired_index = 18
df.insert(desired_index, 'PricePreDiscount', df.pop('PricePreDiscount'))
df['UnitCost'] = ((df['Sales']-df['Profit']) / df['Quantity'])
df.drop(columns='Row ID')

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,...,Category,Sub-Category,Product Name,Quantity,PricePreDiscount,Discount,Sales,Profit,PricePreDiscount_NEW,UnitCost
0,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,Furniture,Bookcases,Bush Somerset Collection Bookcase,2,261.96,0.00,261.9600,41.9136,130.98,110.0232
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",3,731.94,0.00,731.9400,219.5820,243.98,170.7860
2,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,...,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,2,14.62,0.00,14.6200,6.8714,7.31,3.8743
3,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,5,1741.05,0.45,957.5775,-383.0310,348.21,268.1217
4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,2,27.96,0.20,22.3680,2.5164,13.98,9.9258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,...,Furniture,Furnishings,Ultra Door Pull Handle,3,31.56,0.20,25.2480,4.1028,10.52,7.0484
9990,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,2,91.96,0.00,91.9600,15.6332,45.98,38.1634
9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,Technology,Phones,Aastra 57i VoIP phone,2,323.22,0.20,258.5760,19.3932,161.61,119.5914
9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",4,29.60,0.00,29.6000,13.3200,7.40,4.0700


Worksheet Creating DFs and Features

In [7]:
# RENAME COLUMNS
df.rename(columns={'Row ID':'RowID','Order ID':'OrderID','Order Date':'OrderDate','Ship Date':'ShipDate','Ship Mode':'ShipMode'
          ,'Customer ID':'CustomerID','Customer Name':'CustomerName','Postal Code':'PostalCode','Product Name':'ProductName','Product ID':'ProductID','Sub-Category':'SubCategory'}, inplace=True)

# Convert Dates to pd.to_datetime
df['OrderDate'] = pd.to_datetime(df['OrderDate'], format='%m/%d/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
df['ShipDate'] = pd.to_datetime(df['ShipDate'], format='%m/%d/%Y', errors='coerce').dt.strftime('%Y-%m-%d')

# df['OrderDate'] = pd.to_datetime(df['OrderDate'], format='%m/%d/%Y')
# df['ShipDate'] = pd.to_datetime(df['ShipDate'], format='%m/%d/%Y')

#Sort Values based on OrderID and OrderDate
df.sort_values(['OrderID','OrderDate',], inplace=True)

#Create new features for ProfitCalc
df_NEW = df[['ProductID','OrderID','PricePreDiscount_NEW','UnitCost','PricePreDiscount','Discount','Quantity','Sales','Profit']]
df_NEW['PricePreDiscount_NEW'] = (df['Sales'] / (1- df['Discount'])) / df['Quantity']
df_NEW.tail(15)

#Sort DF based on Orderdate and OrderID
df['OrderPosition'] = 0    #--> Do I need this?

#Create a help field that is basically a combination (verketten) of all other adress fields (address_help)
df['address_help'] = ''
for index,row in df.iterrows():
    df.at[index,'address_help'] = (df.at[index,'City']+'-'+str(df.at[index,'PostalCode'])+'-'+df.at[index,'State']+'-'+df.at[index,'State'])
df['AddressID'] = 0
df['Address_ID'] = 0

# Create help fields for TransactionID (Profitcalc) & ShippingID (Delivery)
df['TransactionID'] = ''
df['ShippingID'] = ''

#Create copy of df
df_copy = df.copy() 

C:\Users\rsend\AppData\Local\Temp\ipykernel_18148\4278689958.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_NEW['PricePreDiscount_NEW'] = (df['Sales'] / (1- df['Discount'])) / df['Quantity']


CREATE DF Structure

In [15]:
#Create DF for Customers
df_customer = df_copy[['CustomerID','CustomerName','Segment']].copy()
df_customer.drop_duplicates(subset=['CustomerID'], keep='first', inplace=True)

#Create DF for Products
df_product = df_copy[['ProductID','ProductName','Category','SubCategory']].copy()
df_product.drop_duplicates(subset=['ProductID'], keep='first', inplace=True)

#Create DF for Orders
df_order = df_copy[['OrderID','OrderDate']].copy()
df_order.drop_duplicates(subset=['OrderID'], keep='first', inplace=True)


#Create DF for Profitcalc
df_profitcalc = df_copy[['TransactionID','OrderID','ProductID','OrderDate','ProductName','UnitCost','PricePreDiscount','Discount','Quantity']].copy()


#Create df_common for using for both Address and Delivery and sort by CustomerID&OrderDate
df_common = df_copy[['AddressID','CustomerID','Country','City','State','PostalCode','Region','address_help','OrderDate','ShippingID','ProductID','ProductName','Quantity','ShipMode','ShipDate','Address_ID','OrderID']].copy()
df_common.sort_values(['CustomerID', 'OrderDate'], ascending=False, inplace=True)

# Create a copy of profitcalc (df_profitcalc_incl_profit_u_sales) including profit & Sales and save it separately on destop later for tableau EDA
df_profitcalc_incl_profit_u_sales = df_copy[['TransactionID','OrderID','ProductID','OrderDate','ProductName','UnitCost','PricePreDiscount','Discount','Quantity','Sales','Profit']].copy()
file_path_profitcalc_incl_profit_u_sales = fr'{dropbox_directory}/df_profitcalc_incl_profit_u_sales.csv'
df_profitcalc_incl_profit_u_sales.to_csv(file_path_profitcalc_incl_profit_u_sales)

In [ ]:
# Create a Counter system for each new address:             AddressID
current_customer = None
counter_dict = {}
for index, row in df_common.iterrows():
    if current_customer is None:
        counter = 1
        counter_dict[row['CustomerID']] = counter
    elif current_customer != row['CustomerID']:
        counter = 1
        counter_dict[row['CustomerID']] = counter
    elif current_customer == row['CustomerID'] and (current_help_address != row['address_help'] or current_order_date != row['OrderDate']):
        counter_dict[row['CustomerID']] += 1
    current_customer = row['CustomerID']
    current_help_address = row['address_help']
    current_order_date = row['OrderDate']
    df_common.loc[index, 'Address_ID'] = counter_dict[row['CustomerID']]

# Set AddressID as combination of Address_ID and CustomerID              AddressID    
df_common['Address_ID'] = df_common['Address_ID'].astype(int).astype(str)
df_common['AddressID'] = df_common['CustomerID'] + '-' + df_common['Address_ID']
# df_common.head(50)

In [ ]:
# Define df_Delivery as a copy of df_Address
df_Address = df_common[['AddressID','CustomerID', 'Country', 'City', 'State', 'PostalCode', 'Region']].copy()
df_Address.drop_duplicates(keep='first', inplace=True)

df_Delivery = df_common[['ShippingID','OrderID', 'ProductID','ProductName','Quantity', 'ShipMode', 'CustomerID','OrderDate','ShipDate','AddressID']].copy()

#Edit ShippingID to get an individual markup as theoretically every line in an order with multiple items can represent an individual/different address 
# (like e.g. Amazon sometimes does it with different classification goods, like goods marked as "dangerous" (like hairspray) will only be send to a real
# adress and not to a delivery station )
df_Delivery.sort_values(['OrderID','ProductID'], ascending=False, inplace=True)
current_order_id = None
counter = 1

for index, row in df_Delivery.iterrows():
    if current_order_id != row['OrderID']:
        counter = 1
        current_order_id = row['OrderID']
    df_Delivery.at[index, 'ShippingID'] = row['OrderID'] + '-' + str(counter)
    counter += 1

df_Delivery.tail(15)

In [ ]:
# EDIT df_profitcalc
# Here again every sub-item of an order gets an individual TransactionID. Practically it contains the same information as the DeliveryID because the dataset
# does not give us more distinguished information (very rudimentary address data). Theoretically every individual position in an order should be calculated 
# individually in terms of its effect on profitability

df_profitcalc.sort_values(['OrderID','ProductID'], ascending=False, inplace=True)
current_order_id = None
counter = 1

for index, row in df_profitcalc.iterrows():
    if current_order_id != row['OrderID']:
        counter = 1
        current_order_id = row['OrderID']
    df_profitcalc.at[index, 'TransactionID'] = row['OrderID'] + '-' + str(counter)
    counter += 1

In [ ]:
# remove multiplied combinations of CustomerID
duplicates = df_customer.duplicated(subset=['CustomerID'])
print(df_customer[duplicates])


In [ ]:
# custom_path_4 = 'C:/Users/rsend/Desktop/CAB/Week13_Project_5_Sprint_1/superstore_4.db' (would need to be updated to the path in the config file)

# conn = sqlite3.connect(custom_path_4)
# cursor = conn.cursor()

# # SQL command to clear the table data
# delete_command = "DELETE FROM Customer; " \
#                  "DELETE FROM Product; " \
#                  "DELETE FROM Orders; " \
#                  "DELETE FROM Address; " \
#                  "DELETE FROM ProfitCalc; " \
#                  "DELETE FROM Delivery;"

# # Execute the command
# cursor.executescript(delete_command)

# # Commit the changes
# conn.commit()

# # Close the connection
# conn.close()

In [ ]:
# custom_path_4 = 'C:/Users/rsend/Desktop/CAB/Week13_Project_5_Sprint_1/superstore_4.db'

# conn = sqlite3.connect(custom_path_4)
# cursor = conn.cursor()

# # # Clear the cache
# # cursor.execute('PRAGMA cache_size=-2000;')

# # Table Addresses
# df_customer[[ 'CustomerID', 'CustomerName','Segment']].to_sql( 'Customer', conn, if_exists='append', index=False)
# df_Address[[ 'AddressID','CustomerID', 'Country', 'City', 'State', 'PostalCode', 'Region']].to_sql( 'Address', conn, if_exists='append', index=False)
# df_order.to_sql('Orders', conn, if_exists='append', index= False)
# df_product[[ 'ProductID','ProductName','Category', 'SubCategory',]].to_sql('Product', conn, if_exists='append', index=False)
# df_profitcalc[[ 'TransactionID','OrderID','ProductID','OrderDate','ProductName','Quantity','PricePreDiscount','Discount','UnitCost']].to_sql( 'ProfitCalc', conn, if_exists='append', index=False)
# df_Delivery[[ 'ShippingID','OrderID', 'ProductID','ProductName','Quantity', 'ShipMode', 'CustomerID','OrderDate','ShipDate','AddressID']].to_sql( 'Delivery', conn, if_exists='append', index=False)

# conn.commit()
# conn.close() 